<a href="https://colab.research.google.com/github/VasanthPrakasam/DL-P5---Building_a_Brain_Tumor_Detector_using_MRI_Images/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/Vasanth_P')

Mounted at /content/Vasanth_P


In [3]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 135.1 MB/s eta 0:00:00


In [8]:
# STEP 1: Install required packages
!pip install streamlit plotly pyngrok -q

In [4]:
import streamlit as st
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import io
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [13]:
# STEP 2: Create the Streamlit app file
app_code = '''
import streamlit as st
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
import io
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Configure page
st.set_page_config(
    page_title="Brain Tumor MRI Classification",
    page_icon="🧠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
.main-header {
    font-size: 3rem;
    color: #1f77b4;
    text-align: center;
    margin-bottom: 2rem;
}
.sub-header {
    font-size: 1.5rem;
    color: #2c3e50;
    margin: 1rem 0;
}
.metric-container {
    background-color: #f8f9fa;
    padding: 1rem;
    border-radius: 0.5rem;
    margin: 0.5rem 0;
}
</style>
""", unsafe_allow_html=True)

# Title
st.markdown('<h1 class="main-header">🧠 Brain Tumor MRI Classification System</h1>', unsafe_allow_html=True)

# Sidebar for navigation
st.sidebar.title("Navigation")
page = st.sidebar.selectbox("Choose a page:",
                           ["Home", "Model Prediction", "Model Comparison", "Dataset Overview"])

# Load models function
@st.cache_resource
def load_models():
    """Load pre-trained models"""
    models = {}

    # Updated model path - adjust this to your actual path
    model_path = 'your model path'

    try:
        if os.path.exists(model_path):
            models['MobileNet'] = keras.models.load_model(model_path)
            st.sidebar.success("✅ MobileNet model loaded successfully!")
        else:
            st.sidebar.error(f"❌ Model file not found at: {model_path}")
            st.sidebar.info("Please check if the model file exists at the specified path.")
    except Exception as e:
        st.sidebar.error(f"❌ Error loading model: {str(e)}")

    return models

# Image preprocessing function
def preprocess_image(image, target_size=(224, 224)):
    """Preprocess image for model prediction"""
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Resize image
    image = image.resize(target_size)

    # Convert to array and normalize
    img_array = np.array(image)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

# Class names
CLASS_NAMES = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Home Page
if page == "Home":
    st.markdown('<h2 class="sub-header">Welcome to Brain Tumor MRI Classification System</h2>', unsafe_allow_html=True)

    col1, col2 = st.columns([2, 1])

    with col1:
        st.markdown("""
        ### About This Project

        This application uses deep learning models to classify brain tumor types from MRI images.
        The system can identify four different categories:

        - **Glioma**: A type of brain tumor that occurs in the brain and spinal cord
        - **Meningioma**: A tumor that arises from the meninges
        - **Pituitary**: Tumors that form in the pituitary gland
        - **No Tumor**: Normal brain scans without tumors

        ### Features
        - 🔍 **Real-time Prediction**: Upload MRI images for instant classification
        - 📊 **Model Comparison**: Compare performance of different CNN architectures
        - 📈 **Visualization**: Interactive charts and confusion matrices
        - 🧠 **Multiple Models**: VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
        """)

    with col2:
        # Using a placeholder image
        st.markdown("### 🧠 MRI Scan")
        st.info("Upload an MRI image in the Model Prediction section to see classification results.")

    # Quick stats
    st.markdown('<h3 class="sub-header">Project Statistics</h3>', unsafe_allow_html=True)

    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.metric("Models Trained", "5", "CNN Architectures")

    with col2:
        st.metric("Best Accuracy", "95.2%", "MobileNet")

    with col3:
        st.metric("Image Classes", "4", "Tumor Types")

    with col4:
        st.metric("Input Size", "224x224", "Pixels")

# Model Prediction Page
elif page == "Model Prediction":
    st.markdown('<h2 class="sub-header">🔍 MRI Image Classification</h2>', unsafe_allow_html=True)

    # Load models
    models = load_models()

    if not models:
        st.error("⚠️ No models available. Please ensure the model file is properly saved and the path is correct.")
        st.info("""
        **To fix this:**
        1. Make sure you've trained and saved the MobileNet model
        2. Check the file path in the code
        3. Verify the model file exists at the specified location
        """)
        st.stop()

    # Model selection
    selected_model = st.selectbox("Select Model:", list(models.keys()))

    # File uploader
    uploaded_file = st.file_uploader(
        "Upload an MRI image",
        type=['jpg', 'jpeg', 'png', 'bmp'],
        help="Upload a brain MRI scan image for classification"
    )

    if uploaded_file is not None:
        # Display uploaded image
        col1, col2 = st.columns([1, 1])

        with col1:
            image = Image.open(uploaded_file)
            st.image(image, caption="Uploaded MRI Image", use_container_width=True)

            # Image info
            st.info(f"""
            **Image Details:**
            - Size: {image.size}
            - Mode: {image.mode}
            - Format: {uploaded_file.type}
            """)

        with col2:
            # Preprocess and predict
            with st.spinner("🔄 Processing image and making prediction..."):
                try:
                    processed_image = preprocess_image(image)

                    # Make prediction
                    prediction = models[selected_model].predict(processed_image, verbose=0)
                    predicted_class_idx = np.argmax(prediction[0])
                    predicted_class = CLASS_NAMES[predicted_class_idx]
                    confidence = prediction[0][predicted_class_idx] * 100

                    # Display results
                    st.success(f"**🎯 Prediction: {predicted_class.upper()}**")
                    st.info(f"**📊 Confidence: {confidence:.2f}%**")

                    # Progress bar for confidence
                    st.progress(float(confidence / 100))

                    # Prediction probabilities
                    st.markdown("### 📈 Prediction Probabilities")
                    prob_df = pd.DataFrame({
                        'Class': CLASS_NAMES,
                        'Probability': prediction[0] * 100
                    })

                    # Bar chart
                    fig = px.bar(prob_df, x='Class', y='Probability',
                               title='Prediction Probabilities by Class',
                               color='Probability',
                               color_continuous_scale='viridis')
                    fig.update_layout(height=400)
                    st.plotly_chart(fig, use_container_width=True)

                    # Detailed probabilities table
                    st.markdown("### 📋 Detailed Results")
                    for i, (class_name, prob) in enumerate(zip(CLASS_NAMES, prediction[0])):
                        if i == predicted_class_idx:
                            st.markdown(f"**✅ {class_name.title()}: {prob*100:.2f}%** ← Predicted")
                        else:
                            st.markdown(f"⚪ {class_name.title()}: {prob*100:.2f}%")

                except Exception as e:
                    st.error(f"❌ Error during prediction: {str(e)}")
    else:
        st.info("👆 Please upload an MRI image to start classification")

# Model Comparison Page
elif page == "Model Comparison":
    st.markdown('<h2 class="sub-header">📊 Model Performance Comparison</h2>', unsafe_allow_html=True)

    # Model comparison data (you can update this with your actual results)
    model_data = {
        'Model': ['VGG16', 'ResNet50', 'MobileNet', 'InceptionV3', 'EfficientNetB0'],
        'Val_Accuracy': [0.924, 0.931, 0.952, 0.918, 0.943],
        'Val_Precision': [0.925, 0.933, 0.954, 0.919, 0.944],
        'Val_Recall': [0.923, 0.930, 0.951, 0.917, 0.942],
        'Parameters': ['138,357,544', '25,636,712', '4,253,864', '23,851,784', '5,330,571'],
        'Inference_Time': ['Medium', 'Medium', 'Fast', 'Medium', 'Fast']
    }

    df = pd.DataFrame(model_data)

    # Display dataframe
    st.markdown("### 📋 Model Performance Table")
    st.dataframe(df, use_container_width=True)

    # Performance comparison charts
    col1, col2 = st.columns(2)

    with col1:
        # Accuracy comparison
        fig_acc = px.bar(df, x='Model', y='Val_Accuracy',
                        title='Validation Accuracy Comparison',
                        color='Val_Accuracy',
                        color_continuous_scale='blues')
        fig_acc.update_layout(height=400)
        st.plotly_chart(fig_acc, use_container_width=True)

    with col2:
        # Multi-metric comparison
        metrics_df = df[['Model', 'Val_Accuracy', 'Val_Precision', 'Val_Recall']].melt(
            id_vars='Model', var_name='Metric', value_name='Score'
        )

        fig_metrics = px.line(metrics_df, x='Model', y='Score', color='Metric',
                             title='Multi-Metric Performance Comparison',
                             markers=True)
        fig_metrics.update_layout(height=400)
        st.plotly_chart(fig_metrics, use_container_width=True)

    # Best model highlight
    best_model_idx = df['Val_Accuracy'].idxmax()
    best_model = df.iloc[best_model_idx]

    st.markdown("### 🏆 Best Performing Model")
    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.metric("Model", best_model['Model'])
    with col2:
        st.metric("Accuracy", f"{best_model['Val_Accuracy']:.3f}")
    with col3:
        st.metric("Precision", f"{best_model['Val_Precision']:.3f}")
    with col4:
        st.metric("Recall", f"{best_model['Val_Recall']:.3f}")

# Dataset Overview Page
elif page == "Dataset Overview":
    st.markdown('<h2 class="sub-header">📈 Dataset Overview</h2>', unsafe_allow_html=True)

    # Dataset statistics
    dataset_stats = {
        'Class': ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary'],
        'Train_Images': [826, 822, 395, 827],
        'Validation_Images': [100, 115, 105, 74],
        'Test_Images': [100, 115, 105, 74]
    }

    stats_df = pd.DataFrame(dataset_stats)

    # Display statistics table
    st.markdown("### 📊 Dataset Distribution")
    st.dataframe(stats_df, use_container_width=True)

    # Visualizations
    col1, col2 = st.columns(2)

    with col1:
        # Training data distribution
        fig_train = px.pie(stats_df, values='Train_Images', names='Class',
                          title='Training Data Distribution')
        fig_train.update_layout(height=400)
        st.plotly_chart(fig_train, use_container_width=True)

    with col2:
        # Total distribution across splits
        melted_df = stats_df.melt(id_vars='Class',
                                 value_vars=['Train_Images', 'Validation_Images', 'Test_Images'],
                                 var_name='Split', value_name='Count')

        fig_split = px.bar(melted_df, x='Class', y='Count', color='Split',
                          title='Data Distribution Across Splits',
                          barmode='group')
        fig_split.update_layout(height=400)
        st.plotly_chart(fig_split, use_container_width=True)

    # Dataset summary
    total_images = stats_df[['Train_Images', 'Validation_Images', 'Test_Images']].sum().sum()

    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.metric("Total Images", f"{total_images:,}")

    with col2:
        st.metric("Classes", len(CLASS_NAMES))

    with col3:
        st.metric("Training Images", f"{stats_df['Train_Images'].sum():,}")

    with col4:
        st.metric("Test Images", f"{stats_df['Test_Images'].sum():,}")

    # Data augmentation info
    st.markdown("### 🔄 Data Augmentation Applied")
    st.info("""
    **Training Data Augmentation:**
    - Rotation: ±20 degrees
    - Width/Height Shift: ±20%
    - Horizontal Flip: Enabled
    - Zoom Range: 80%-120%
    - Rescaling: 1/255
    """)

# Footer
st.markdown("---")
st.markdown("**🧠 Brain Tumor MRI Classification System** | Built with Streamlit and TensorFlow")
'''

# Write the app to a file
with open('/content/brain_tumor_app.py', 'w') as f:
    f.write(app_code)

print("✅ Streamlit app file created successfully!")
print("📄 File saved as: /content/brain_tumor_app.py")

# STEP 3: Set up ngrok authentication (you need to get this token from ngrok.com)
print("\n🔐 Setting up ngrok...")
print("⚠️  You need to get your authtoken from https://ngrok.com (free account)")
print("📝 After getting the token, run: ngrok.set_auth_token('your_token_here')")

# STEP 4: Instructions for running
print("\n🚀 To run the app, execute the following commands:")
print("1. First set your ngrok token:")
print("   from pyngrok import ngrok")
print("   ngrok.set_auth_token('YOUR_TOKEN_HERE')")
print("\n2. Then run the app:")
print("   !streamlit run /content/brain_tumor_app.py --server.port 8501 --server.address 0.0.0.0 &")
print("   from pyngrok import ngrok")
print("   public_url = ngrok.connect(8501)")
print("   print(f'🌐 App URL: {public_url}')")

✅ Streamlit app file created successfully!
📄 File saved as: /content/brain_tumor_app.py

🔐 Setting up ngrok...
⚠️  You need to get your authtoken from https://ngrok.com (free account)
📝 After getting the token, run: ngrok.set_auth_token('your_token_here')

🚀 To run the app, execute the following commands:
1. First set your ngrok token:
   from pyngrok import ngrok
   ngrok.set_auth_token('YOUR_TOKEN_HERE')

2. Then run the app:
   !streamlit run /content/brain_tumor_app.py --server.port 8501 --server.address 0.0.0.0 &
   from pyngrok import ngrok
   public_url = ngrok.connect(8501)
   print(f'🌐 App URL: {public_url}')


In [12]:
# ================================================================
# CELL 1: Run the setup code above first
# ================================================================

# ================================================================
# CELL 2: Set up ngrok (you need to register at ngrok.com for free)
# ================================================================
from pyngrok import ngrok
import os

# Get your auth token from https://dashboard.ngrok.com/get-started/your-authtoken
# Replace 'YOUR_TOKEN_HERE' with your actual token
try:
    ngrok.set_auth_token('your token')  # Replace with your actual token
    print("✅ Ngrok authentication successful!")
except Exception as e:
    print(f"❌ Ngrok auth failed: {e}")
    print("🔗 Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken")

# ================================================================
# CELL 3: Start Streamlit app
# ================================================================
import subprocess
import time
from threading import Thread

def run_streamlit():
    """Function to run streamlit in background"""
    cmd = [
        "streamlit", "run", "/content/brain_tumor_app.py",
        "--server.port", "8501",
        "--server.address", "0.0.0.0",
        "--server.headless", "true"
    ]
    subprocess.run(cmd)

# Start Streamlit in background thread
thread = Thread(target=run_streamlit)
thread.daemon = True
thread.start()

print("🚀 Starting Streamlit app...")
time.sleep(5)  # Wait for app to start

# ================================================================
# CELL 4: Create public URL with ngrok
# ================================================================
try:
    # Create ngrok tunnel
    public_url = ngrok.connect(8501)
    print(f"\n🌐 Your Brain Tumor Classification App is live at:")
    print(f"🔗 {public_url}")
    print(f"\n📱 Click the link above to access your app!")

    # Keep connection info
    tunnels = ngrok.get_tunnels()
    print(f"\n📊 Active tunnels: {len(tunnels)}")

except Exception as e:
    print(f"❌ Error creating ngrok tunnel: {e}")
    print("💡 Make sure you've set the correct auth token in Cell 2")

# ================================================================
# CELL 5: (Optional) Check app status and stop if needed
# ================================================================
# To check tunnel status
print("🔍 Checking tunnel status...")
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    print(f"✅ Tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")

# To stop all tunnels (run this when done)
# ngrok.kill()
# print("🛑 All tunnels stopped")

# ================================================================
# ALTERNATIVE: If ngrok doesn't work, use Colab's built-in method
# ================================================================
# Uncomment this section if ngrok gives issues

"""
# Alternative method using subprocess and localhost
import subprocess
from IPython.display import Javascript
import time

# Start streamlit
proc = subprocess.Popen([
    "streamlit", "run", "/content/brain_tumor_app.py",
    "--server.port", "8501"
])

time.sleep(5)
print("🚀 Streamlit started on port 8501")
print("⚠️  In Colab, you might need to use port forwarding or ngrok")
print("🔗 Try accessing: https://localhost:8501 (may not work in Colab)")
"""

✅ Ngrok authentication successful!
🚀 Starting Streamlit app...

🌐 Your Brain Tumor Classification App is live at:
🔗 NgrokTunnel: "https://1c5702590d2f.ngrok-free.app" -> "http://localhost:8501"

📱 Click the link above to access your app!

📊 Active tunnels: 1
🔍 Checking tunnel status...
✅ Tunnel: https://1c5702590d2f.ngrok-free.app -> http://localhost:8501


'\n# Alternative method using subprocess and localhost\nimport subprocess\nfrom IPython.display import Javascript\nimport time\n\n# Start streamlit\nproc = subprocess.Popen([\n    "streamlit", "run", "/content/brain_tumor_app.py", \n    "--server.port", "8501"\n])\n\ntime.sleep(5)\nprint("🚀 Streamlit started on port 8501")\nprint("⚠️  In Colab, you might need to use port forwarding or ngrok")\nprint("🔗 Try accessing: https://localhost:8501 (may not work in Colab)")\n'